<span style="color:blue">Copyright (c) 2014-2019 National Technology and Engineering
Solutions of Sandia, LLC. Under the terms of Contract DE-NA0003525
with National Technology and Engineering Solutions of Sandia, LLC,
the U.S. Government retains certain rights in this software.</span>    
    
<span style="color:blue">Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions
are met:</span>    
    
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:green">1. Redistributions of source code must retain the above copyright
   notice, this list of conditions and the following disclaimer.</span>    
    
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:green">2. Redistributions in binary form must reproduce the above copyright
   notice, this list of conditions and the following disclaimer in the
   documentation and/or other materials provided with the distribution.</span>    
    
<span style="color:blue">THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY
THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

**Purpose:** Sample code to assemble points into trajectories    
When you read in points from a file, you need to build those points into trajectories. As we know, data can be noisy, therefore we want to filter our trajectories to have a minimum number of points and/or a certain time or distance between points in a trajectory. The trajectory builder does all of this. 

Imports

In [37]:
from tracktable.domain.terrestrial import TrajectoryPointReader        # Read points from file
from tracktable.source.trajectory import AssembleTrajectoryFromPoints  # Turn points into trajectories

**Requirements:** In order to use the trajectory builder, we need a source that gives us data points in time sequential order. As with before, we can use the Point Reader.

In [38]:
inFile = open('../data/SampleFlightsUS.csv')
reader = TrajectoryPointReader()
reader.input = inFile
reader.comment_character = '#'
reader.field_delimiter = ','
# Set columns for data we care about
reader.object_id_column = 0
reader.timestamp_column = 1
reader.coordinates[0] = 2
reader.coordinates[1] = 3
reader.set_real_field_column('Heading', 5)

Now that we have our points ready to be read, we need to create a builder to take those points and form trajectories.    
    
For now, we are going to leave the parameters at default. 

In [39]:
builder = AssembleTrajectoryFromPoints()
builder.input = reader
builder.minumum_length = 3                  # An example of how to set the minimum number of points for each trajectory. Default is 2.

That's it! Now we can play with the trajectories. As before with the point reader by itself, no data is loaded until you interact with the data.    
    
Let's start with seeing how many trajectories we have. We can do this, and see what they look like by dumping the trajectories to a list.  

In [40]:
traj = list(builder.trajectories())
print(len(traj))

INFO: Trajectory assembly: New trajectories will be declared after a separation of None units or 0:30:00 seconds.
INFO: Trajectories with fewer than 2 points will be rejected.
INFO: Done assembling trajectories.
29


Looking at our first trajectory, we can see all the points included. See how they are in time sequential order and share the same object_id, which is what we would expect from a trajectory. 

In [41]:
print(*traj[0])

[JJJ010@ 2004-12-10 14:14:40: (-111.501, 47.3697) Properties: ( {Heading [real]: 134})] [JJJ010@ 2004-12-10 14:15:41: (-111.528, 47.3353) Properties: ( {Heading [real]: 134})] [JJJ010@ 2004-12-10 14:16:43: (-111.551, 47.3039) Properties: ( {Heading [real]: 118})] [JJJ010@ 2004-12-10 14:17:45: (-111.575, 47.2719) Properties: ( {Heading [real]: 121})] [JJJ010@ 2004-12-10 14:18:47: (-111.599, 47.2403) Properties: ( {Heading [real]: 120})] [JJJ010@ 2004-12-10 14:19:49: (-111.621, 47.2081) Properties: ( {Heading [real]: 120})] [JJJ010@ 2004-12-10 14:20:50: (-111.646, 47.1761) Properties: ( {Heading [real]: 124})] [JJJ010@ 2004-12-10 14:21:52: (-111.676, 47.1375) Properties: ( {Heading [real]: 147})] [JJJ010@ 2004-12-10 14:22:54: (-111.699, 47.1036) Properties: ( {Heading [real]: 126})] [JJJ010@ 2004-12-10 14:23:56: (-111.726, 47.0692) Properties: ( {Heading [real]: 132})] [JJJ010@ 2004-12-10 14:24:58: (-111.752, 47.0336) Properties: ( {Heading [real]: 134})] [JJJ010@ 2004-12-10 14:26:01: (-

Here is an example of how to access specific information within a trajectory. We show how to get:    
&emsp; 1. Core data such as object_id    
&emsp; 2. Longitude and Latitude    
&emsp; 3. Custom columns such as 'Heading'    

In [42]:
print('Object_id: ' + traj[0][0].object_id)                     # Trajectory 0 in list, point 0 in trajectory
print('Latitude: ' + str(traj[0][0][1]))                        # Trajectory 0 in list, point 0 in trajectory, coordinate[1]
print('Heading: ' + str(traj[0][0].properties['Heading']))      

Object_id: JJJ010
Latitude: 47.3697
Heading: 134.0


In [43]:
inFile.close()

The builder has multiple parameters that can be changed. For example, we can    
&emsp; 1. Change the minimum number of points needed to form a trajectory. (builder.minimum_length = 10)    
&emsp; 2. Change the minimum distance between points. (builder.separation_stance = 100)    
&emsp; 3. Change the minimum time between points. (builder.separation_time = datetime.timedelta(minutes=30))    